# Segmenting and Clustering Neighborhood in Toronto Part 2

In [24]:
!pip install geopandas
!pip install geopy
!pip install geocoder

In [8]:
import pandas as pd
import numpy as np
import geocoder # import geocoder
print('Imported')

Imported


In [9]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df.describe()

,Latitude,Longitude
count,103.000000,103.000000
mean,43.704608,-79.397153
std,0.052463,0.097146
min,43.602414,-79.615819
25%,43.660567,-79.464763
50%,43.696948,-79.388790
75%,43.745320,-79.340923
max,43.836125,-79.160497


In [11]:
!pip install geocoder

In [12]:

def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.709020000000066, -79.36348999999996]

In [13]:
# Retrieving Postal Code Co-ordinates
postal_codes = df['Postal Code']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

In [14]:
# Adding Columns Latitude & Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [15]:
df[df['Postal Code'] == 'M5G']

,Postal Code,Latitude,Longitude
57,M5G,43.65609,-79.38493


In [16]:
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.81139,-79.19662
1,M1C,43.78574,-79.15875
2,M1E,43.76575,-79.17470
3,M1G,43.76812,-79.21761
4,M1H,43.76944,-79.23892


In [17]:
import requests
from bs4 import BeautifulSoup
print("Imported!")


Imported!


In [18]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data, 'lxml')

In [19]:
table_contents=[]
table=wiki_data.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df1=pd.DataFrame(table_contents)
df1['Borough']=df1['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [20]:
df1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [25]:
df1_coords = pd.DataFrame(coords, columns = ['Latitude', 'Longitude'])
df1['Latitude'] = df1_coords['Latitude']
df1['Longitude'] = df1_coords['Longitude']
df1

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.81139,-79.19662
1,M4A,North York,Victoria Village,43.78574,-79.15875
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.76575,-79.17470
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.76812,-79.21761
4,M7A,Queen's Park,Ontario Provincial Government,43.76944,-79.23892
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.70507,-79.51804
99,M4Y,Downtown Toronto,Church and Wellesley,43.69630,-79.52926
100,M7Y,East Toronto Business,Enclave of M4L,43.68681,-79.55728
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.74453,-79.58624


In [22]:
df1[df1.PostalCode == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
24,M5G,Downtown Toronto,Central Bay Street,43.77989,-79.44678


In [23]:
df1.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.81139,-79.19662
1,M4A,North York,Victoria Village,43.78574,-79.15875
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.76575,-79.17470
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.76812,-79.21761
4,M7A,Queen's Park,Ontario Provincial Government,43.76944,-79.23892
5,M9A,Etobicoke,Islington Avenue,43.74446,-79.23117
6,M1B,Scarborough,"Malvern, Rouge",43.72582,-79.26461
7,M3B,North York,Don Mills North,43.71289,-79.28506
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.72360,-79.23496
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.69510,-79.26466
